# Semantic Search with Fine Tuning Model

In the Module 3 and Module 4, we are using pretrained BERT model to convert text into vector. However, to improve the model accuary, we need to fine tune the BERT model. See [Fine-tune a pretrained model](https://huggingface.co/docs/transformers/training) for more informaiton.

In this module, we will use [Amazon Product Question and Answer (PQA) dataset](https://registry.opendata.aws/amazon-pqa/) to fine tune the model. The architecture of this module is like follows:

![semantic_search_with_fine_tuning](semantic_search_with_fine_tuning.png)


### 1.Import PyTorch and check version.

As in the previous modules, let's import PyTorch and confirm that have have the latest version of PyTorch. The version should already be 1.10.2 or higher.

In [ ]:
import torch
print(torch.__version__)

If the PyTorch version is 1.10.2 or higher, you can skip to step 2.

If the PyTorch version is not 1.10.2 or higher, we need to update PyTorch and restart the kernel.

In [ ]:
!pip install --upgrade torch

In [ ]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
restartkernel()

Let's recheck the version of Torch to ensure everything is up to date. The version should be 1.10.2 or higher.

In [ ]:
import torch
print(torch.__version__)

### 2. Install additional libraries

Now let's install some additional libraries we'll need later on.

In [ ]:
!pip install -U sentence-transformers rank_bm25
!pip install -q opensearch-py
!pip install -q tqdm

### 3. Get Cloud Formation stack output variables

We also need to grab some key values from the infrastructure we provisioned using CloudFormation. To do this, we will list the outputs from the stack and store this in "outputs" to be used later.

You can ignore any "PythonDeprecationWarning" warnings.

In [ ]:
import boto3

cfn = boto3.client('cloudformation')

def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "semantic-search"

outputs = get_cfn_outputs(cloudformation_stack_name)

bucket = outputs['s3BucketTraining']
aos_host = outputs['OpenSearchDomainEndpoint']

outputs

### 4. Copy the data set locally
Before we can run any queries, we need to download the Amazon Product Question and Answer data from : https://registry.opendata.aws/amazon-pqa/

Let's start by having a look at all the files in the dataset.

In [ ]:
!aws s3 ls --no-sign-request s3://amazon-pqa/

There are a lot of files here, so for the purposes of this demo, we focus on just the headset data. Let's download the amazon_pqa_headsets.json data locally.

In [ ]:
!aws s3 cp --no-sign-request s3://amazon-pqa/amazon_pqa_headsets.json ./amazon-pqa/amazon_pqa_headsets.json

Cout total records in the dataset.

In [ ]:
!wc -l ./amazon-pqa/amazon_pqa_headsets.json

### 5. Prepare training dataset


Use [sentence transformer](https://www.sbert.net/docs/training/overview.html) to train the model. The input of training data is:
* pair of text/sentence
* label to indicate the semantic similarity between the sentences

For demo purpose, we set orginal "question" and "answer" pair semantic similarity to "1.0". The "question" and "answer in previous row" semantic similarity to "0.0". In production, we shall use our training dataset based on business requirements.

To reduce data load time, we only load 10,000 rows data for training.


In [ ]:
import json
import pandas as pd

def load_pqa(file_name,number_rows=10000):
    qa_list = []
    df = pd.DataFrame(columns=('question', 'answer','label'))
    with open(file_name) as f:
        i=0
        previous_row_data = None
        for line in f:
            data = json.loads(line)
            df.loc[i] = [data['question_text'],data['answers'][0]['answer_text'],1.0]
            i+=1
            if previous_row_data is not None:
                df.loc[i] = [data['question_text'],previous_row_data['answers'][0]['answer_text'],0.0]
            previous_row_data = data
            i+=1
            if(i == number_rows*2):
                break
    return df


qa_list = load_pqa('amazon-pqa/amazon_pqa_headsets.json')

We split the dataset into training dataset, validation dataset and test dataset.

In [ ]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
from sklearn.model_selection import train_test_split
from sentence_transformers.readers import InputExample

train_set,test_set = train_test_split(qa_list,test_size=0.2,shuffle=True)
training_set, validation_set = train_test_split(train_set,test_size=0.2)

def create_input_sample(data_set):
    train_samples = []
    for index,row in data_set.iterrows():
        input_example = InputExample(texts=[row['question'], row['answer']], label=row['label'])
        train_samples.append(input_example)
    return train_samples

training_samples = create_input_sample(training_set)
validation_samples = create_input_sample(validation_set)
test_samples = create_input_sample(test_set)


### 6. Train the model

#### Note: It will take more than 20 minutes to complete the training.

In [ ]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

model_name = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
train_batch_size = 16
num_epochs = 1
model_save_path = 'output/fine_tuned_'+model_name.replace("/", "-")

word_embedding_model = models.Transformer(model_name)

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_dataloader = DataLoader(training_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

evaluator = EmbeddingSimilarityEvaluator.from_input_examples(validation_samples, name='pqa-valucation')


warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up



# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)


model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='pqa-test')
test_evaluator(model, output_path=model_save_path)

### 7. Upload the fine-tuned model into S3

Create tar file for the model

In [ ]:
!cd output/fine_tuned_sentence-transformers-distilbert-base-nli-stsb-mean-tokens && tar czvf ../model.tar.gz *

Upload the model to S3

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

inputs = sagemaker_session.upload_data(path='output/model.tar.gz', key_prefix='fine-tuned-transformers-model')
inputs

### 8. Deploy the BERT model to SageMaker Endpoint

First we need to create a PyTorchModel object. The deploy() method on the model object creates an endpoint which serves prediction requests in real-time. If the instance_type is set to a SageMaker instance type (e.g. ml.m5.large) then the model will be deployed on SageMaker. If the instance_type parameter is set to local then it will be deployed locally as a Docker container and ready for testing locally.

First we need to create a Predictor class to accept TEXT as input and output JSON. The default behaviour is to accept a numpy array.


In [ ]:
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

class StringPredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')


Deploy the BERT model to Sagemaker Endpoint

#### Note: This process will take serveral minutes to complete.

In [ ]:
import time

pytorch_model = PyTorchModel(model_data = inputs, 
                             role=role, 
                             entry_point ='inference.py',
                             source_dir = './code',
                             py_version = 'py38', 
                             framework_version = '1.10.2',
                             predictor_cls=StringPredictor)

predictor = pytorch_model.deploy(instance_type='ml.m5d.large', 
                                 initial_instance_count=1, 
                                 endpoint_name = f'semantic-search-model-{int(time.time())}')

### 9. Test the SageMaker Endpoint.

Now SageMaker Endpoind is ready, we can test the endpoind. Same as before, input is text data, output is vector data. The difference is the BERT model is fune tuned with our business domain data.

In [ ]:
import json
original_payload = 'Does this work with xbox?'
features = predictor.predict(original_payload)
vector_data = json.loads(features)

vector_data


### 10. Create an Amazon OpenSearch Service cluster connection.
Next, we'll use Python API to set up connection with Amazon OpenSearch Service Cluster.

Note: if you're using a region other than us-east-1, please update the region in the code below. 

Use Python API to set up connection with Amazon OpenSearch Cluster

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3

region = 'us-east-1' 

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)
index_name = 'nlp_pqa'

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

### 11. Create a index in Amazon OpenSearch Service
Whereas we previously created an index with 2 fields, this time we'll define the index with 3 fields: the first field ' question_vector' holds the vector representation of the question, the second is the "question" for raw sentence and the third field is "answer" for the raw answer data.

To create the index, we first define the index in JSON, then use the aos_client connection we initiated ealier to create the index in OpenSearch.

In [ ]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil",
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "question_vector": {
                "type": "knn_vector",
                "dimension": 768,
                "store": True
            },
            "question": {
                "type": "text",
                "store": True
            },
            "answer": {
                "type": "text",
                "store": True
            }
        }
    }
}


If for any reason you need to recreate your dataset, you can uncomment and execute the following to delete any previously created indexes. If this is the first time you're running this, you can skip this step.

In [ ]:
#aos_client.indices.delete(index="nlp_pqa")


Using the above index definition, we now need to create the index in Amazon OpenSearch Service

In [ ]:
aos_client.indices.create(index="nlp_pqa",body=knn_index,ignore=400)


Let's verify the created index information

In [ ]:
aos_client.indices.get(index="nlp_pqa")

### 12. Load the raw data into the Index
Next, let's load the headset enhanced PQA data into the index we've just created.

In [ ]:
import json
from tqdm.contrib.concurrent import process_map
from multiprocessing import cpu_count


def load_pqa_as_json(file_name,number_rows=1000):
    result=[]
    with open(file_name) as f:
        i=0
        for line in f:
            data = json.loads(line)
            result.append(data)
            i+=1
            if(i == number_rows):
                break
    return result


qa_list_json = load_pqa_as_json('amazon-pqa/amazon_pqa_headsets.json',number_rows=1000)


def es_import(question):
    vector = json.loads(predictor.predict(question["question_text"]))
    aos_client.index(index='nlp_pqa',
             body={"question_vector": vector, "question": question["question_text"],"answer":question["answers"][0]["answer_text"]}
            )
        
workers = 4 * cpu_count()
    
process_map(es_import, qa_list_json, max_workers=workers,chunksize=1000)

To validate the load, we'll query the number of documents number in the index. We should have 1000 hits in the index.

In [ ]:
res = aos_client.search(index="nlp_pqa", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])

### 13. Generate vector data for user input query 

Next, we'll use SageMaker Endpoint to convert our input question "does this work with xbox?" into a vector. 

In [ ]:
query_raw_sentences = ['does this work with xbox?']
client = boto3.client('sagemaker-runtime')
ENDPOINT_NAME = predictor.endpoint
response = client.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                       ContentType='text/plain',
                                       Body=query_raw_sentences[0])

search_vector = json.loads((response['Body'].read()))


### 14. Search vector data with "Semantic Search" 

Now that we have vector data in Amazon OpenSearch Service and a vector for our query question, let's perform a KNN search in Amazon OpenSearch.


In [ ]:

query={
    "size": 50,
    "query": {
        "knn": {
            "question_vector":{
                "vector":search_vector,
                "k":50
            }
        }
    }
}

res = aos_client.search(index="nlp_pqa", 
                       body=query,
                       stored_fields=["question","answer"])
#print("Got %d Hits:" % res['hits']['total']['value'])
query_result=[]
for hit in res['hits']['hits']:
    row=[hit['_id'],hit['_score'],hit['fields']['question'][0],hit['fields']['answer'][0]]
    query_result.append(row)

query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","question","answer"])
display(query_result_df)

### 15. Search the same query with "Text Search"

Let's repeat the same query with a keyword search and compare the differences.

In [ ]:
query={
    "size": 50,
    "query": {
        "match": {
            "question":"does this work with xbox?"
        }
    }
}

res = aos_client.search(index="nlp_pqa", 
                       body=query,
                       stored_fields=["question","answer"])
#print("Got %d Hits:" % res['hits']['total']['value'])
query_result=[]
for hit in res['hits']['hits']:
    row=[hit['_id'],hit['_score'],hit['fields']['question'][0],hit['fields']['answer'][0]]
    query_result.append(row)

query_result_df = pd.DataFrame(data=query_result,columns=["_id","_score","question","answer"])
display(query_result_df)


### 16. Observe The Results

Compare the first few records in the two searches above. For the Semantic search, the first 10 or so results are very similar to our input questions, as we expect. Compare this to keyword search, where the results quickly start to deviate from our search query (e.g. "it shows xbox 360. Does it work for ps3 as well?" - this matches on keywords but has a different meaning).